<a href="https://colab.research.google.com/github/StasGT/Pedestrian-Tracking/blob/new/ByteTrack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Use GPU enviroment please.**

In [ ]:
!git clone https://github.com/ifzhang/ByteTrack.git
%cd ByteTrack
!ls
!pip3 install -r requirements.txt
!python3 setup.py develop

running develop
/usr/local/lib/python3.11/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog

In [8]:
#!pip install loguru
#!pip install thop
#!pip install lap
#!pip install cython_bbox
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 103.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.8 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
blosc2 3.5.0 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
bigframes 2.8.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is inco

In [ ]:

import numpy as np
import torchvision.transforms as T

import argparse
import os
import os.path as osp
import time
import cv2
import torch

from loguru import logger

from yolox.data.data_augment import preproc
from yolox.exp import get_exp
from yolox.utils import fuse_model, get_model_info, postprocess
from yolox.utils.visualize import plot_tracking
from yolox.tracker.byte_tracker import BYTETracker
from yolox.tracking_utils.timer import Timer

!nvcc --version

/content/ByteTrack/yolox/tracker/kalman_filter.py:3: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.0)
  import scipy.linalg


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


Necessary to load weights - https://drive.google.com/file/d/1P4mY0Yyd3PPTybgZkjMYhFri88nTmJX5/view?usp=sharing

And put it in the ByteTrack folder

In [10]:
%cd ByteTrack
!ls
!python tools/demo_track.py video -f exps/example/mot/yolox_x_mix_det.py -c bytetrack_x_mot17.pth.tar --fp16 --fuse --save_result



[Errno 2] No such file or directory: 'ByteTrack'
/content/ByteTrack
assets			   Dockerfile	     setup.cfg	yolox
build			   exps		     setup.py	yolox.egg-info
bytetrack_x_mot17.pth.tar  LICENSE	     tools	YOLOX_outputs
datasets		   README.md	     tutorials
deploy			   requirements.txt  videos
2025-06-28 07:17:11.986 | INFO     | __main__:main:314 - Args: Namespace(demo='video', experiment_name='yolox_x_mix_det', name=None, path='./videos/palace.mp4', camid=0, save_result=True, exp_file='exps/example/mot/yolox_x_mix_det.py', ckpt='bytetrack_x_mot17.pth.tar', device=device(type='cuda'), conf=None, nms=None, tsize=None, fps=30, fp16=True, fuse=True, trt=False, track_thresh=0.5, track_buffer=30, match_thresh=0.8, aspect_ratio_thresh=1.6, min_box_area=10, mot20=False)
/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/Ten

**Input video is in the folder: ByteTrack/videos/**

**Output video in: ByteTrack/YOLOX_outputs/yolox_x_mix_det/track_vis/**